In [27]:
import pennylane as qml
from pennylane import numpy as np

### Overview
In this notebook I manually work through the global cost function. We start with this simple problem:

$$
X\ket{x} = \ket{0}
$$

We look at a specific instance of the problem where:
$$V(\alpha) = R_y(\pi/2) = \frac{1}{\sqrt{2}}\begin{bmatrix}1 & -1 \\ 1 & 1\end{bmatrix}$$
So 
$$
\ket{\psi} = V(\alpha)\ket{0} = \ket{+}
$$
We have 
$$
C_G = 1 - |\braket{b|\Psi}|^2
$$
Where
$$
|\braket{b|\Psi}|^2 = \frac{|\braket{b|\psi}|^2}{\braket{\psi|\psi}}
$$
Focusing on the top term, we get:
$$
|\braket{b|\psi}|^2 = \sum_{ll'}c_lc_{l'}* \gamma_{ll'}
$$
In our case, $l = l' = 1$ and $c_l = 1$, so 
$$
= \gamma_{11}
$$

Now, 
$$
\gamma_{ll'} = \braket{0|U^\dag A_l V|0}\braket{0|V^\dag A_{l'}^\dag U|0} \\
= \braket{0|I X R_y(\pi/2)|0}\braket{0|R_y(\pi/2)^\dag X I|0} \\
= |\braket{0|+}|^2 \\
= \frac{1}{\sqrt{2}}
$$

So we should expect to see $\frac{1}{\sqrt{2}}$ as the output of the overlap test.

In [28]:
n_qubits = 1
ancilla_idx = n_qubits*2

In [29]:
dev_mu = qml.device("default.qubit", wires=n_qubits+1)
dev_gamma = qml.device("default.qubit", wires=n_qubits*2+1)

In [30]:
def variational_block(offset=0):
    # gives |x> = |0+>
    qml.RY(np.pi/2, wires=offset)

def U_b(offset=0):
    # gives |b> = |00>
    pass

def CA(offset=0):
    qml.CNOT(wires=(ancilla_idx, offset))

In [31]:
@qml.qnode(dev_gamma)
def hadamard_overlap_test(weights, l=None, lp=None, part=None):
    """implements the overlap test for C_G"""

    # H on ancilla index
    qml.Hadamard(ancilla_idx)

    # Variational circuit generating a guess for the solution vector |x> applied to the top half
    variational_block(offset=n_qubits)

    # unitary U_b associated to the problem vector |b> applied to the bottom half
    # In this specific example Adjoint(U_b) = U_b.
    U_b()

    # Controlled application of the unitary component A_l of the problem matrix A on the top half.
    CA(offset=n_qubits)

    # Controlled application of Adjoint(A_lp) applied to the bottom half
    # In this specific example Adjoint(A_lp) = A_lp. #TODO: is it really?
    CA()

    if part == "Im":
        qml.RZ(phi=-np.pi/2, wires=ancilla_idx)

    # bell basis observable
    [qml.CNOT(wires=(i+n_qubits, i)) for i in range(n_qubits)]
    [qml.Hadamard(wires=i) for i in range(n_qubits, n_qubits*2 + 1)]

    # to get P(0) - P(1) we need to perform linear classical post-processing which involves using the probabilities
    return qml.probs(wires=range(n_qubits*2 + 1))

def cost_global(problem, weights, local_hadamard_test, hadamard_overlap_test):
    """Global version of the cost function. Tends to zero when A|x> is proportional to |b>."""

    c, _ = problem.get_coeffs()

    norm = 0.0
    overlap = 0.0

    for l in range(0, len(c)):
        for lp in range(0, len(c)):
            # start = time.time()
            norm = norm + c[l] * np.conj(c[lp]) * mu(weights, local_hadamard_test, problem, l, lp, -1)
            # print(f"norm accum ({l*len(c) + lp})")

            overlap = overlap + c[l] * np.conj(c[lp]) * gamma(weights, hadamard_overlap_test, problem, l, lp)

    norm = abs(norm)
    overlap = abs(overlap)

    return 1 - overlap / norm # TODO: double check this expression

@qml.qnode(dev_mu)
def local_hadamard_test(weights, l=None, lp=None, j=None, part=None):
    """this function implements the local hadamard test for calculating mu and the norm"""
    ancilla_idx = n_qubits

    # First Hadamard gate applied to the ancillary qubit.
    qml.Hadamard(wires=ancilla_idx)

    # For estimating the imaginary part of the coefficient "mu", we must add a "-i"
    # phase gate.
    if part == "Im" or part == "im":
        qml.PhaseShift(-np.pi / 2, wires=ancilla_idx)

    # Variational circuit generating a guess for the solution vector |x>
    variational_block()

    # Controlled application of the unitary component A_l of the problem matrix A.
    CA()

    # Adjoint of the unitary U_b associated to the problem vector |b>.
    # In this specific example Adjoint(U_b) = U_b.
    U_b()

    # Controlled Z operator at position j. If j = -1, apply the identity.
    if j != -1:
        qml.CZ(wires=[ancilla_idx, j])

    # Unitary U_b associated to the problem vector |b>.
    U_b()

    # Controlled application of Adjoint(A_lp).
    # In this specific example Adjoint(A_lp) = A_lp.
    CA()

    # Second Hadamard gate applied to the ancillary qubit.
    qml.Hadamard(wires=ancilla_idx)

    # Expectation value of Z for the ancillary qubit.
    return qml.expval(qml.PauliZ(wires=ancilla_idx))


def get_bin(state: int, n_qubits):
    """
    Helper function that identifies the correct bin for the overlap test. Details can be found in Cincio et. al

    @param
    state: a measurement outcome as an int 
    return: (-1 or 1, corresponding to whether the prob on the bitstring should be added or subtracted)
    """
    acc = 1

    # if aux qubit is 1
    if state & 2**(n_qubits*2):
        acc *= -1

    for i in range(n_qubits):
        if state & 2**i and state & 2**(i + n_qubits):
            acc *= -1

    return acc

# Computes the mu coefficients
def mu(weights, problem, l=None, lp=None, j=None):
    """Generates the coefficients to compute the "local" cost function C_L."""

    # start = time.time()
    mu_real = local_hadamard_test(weights, problem, l=l, lp=lp, j=j, part="Re")
    mu_imag = local_hadamard_test(weights, problem, l=l, lp=lp, j=j, part="Im")
    # print(f"mu: {time.time() - start:.2f}")

    return mu_real + 1.0j * mu_imag

def gamma(weights, l=None, lp=None):
    """calculates the gamma coefficients for C_G"""

    probs_real = hadamard_overlap_test(weights, l=l, lp=lp, part="Re")
    probs_imag = hadamard_overlap_test(weights, l=l, lp=lp, part="Im")

    gamma_real = 0
    gamma_imag = 0

    # I have a feeling a lot of these are cancelling each other out resulting in a very low output value
    for state, prob in enumerate(probs_real):
        gamma_real += get_bin(state, n_qubits) * prob
    
    for state, prob in enumerate(probs_imag):
        gamma_imag += get_bin(state, n_qubits) * prob

    # print(f"gamma: {time.time() - start:.2f}")

    return 2 * (gamma_real + 1.0j * gamma_imag) # see appendix C for the 2x coeff

In [32]:
hadamard_overlap_test([], 0, 0, "Re")

tensor([0.25, 0.  , 0.  , 0.25, 0.25, 0.  , 0.  , 0.25], requires_grad=True)

In [34]:
hadamard_overlap_test([], 0, 0, "Im")

tensor([0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125], requires_grad=True)

Doing the classical post-processing by hand, we get:

$$C(0) = P(000) + P(001) P(010) - P(011) = 0.25 + 0 + 0 - 0.25 = 0 \\
C(1) = P(100) + P(101) P(110) - P(111) = 0.25 + 0 + 0 - 0.25 = 0 \\
Re[\gamma_{11}] = 2(C(0) - C(1)) = 0
$$
Imaginary:
$$C(0) = P(000) + P(001) P(010) - P(011) = 0.125 + 0.125 + 0.125 - 0.125 = 0.25 \\
C(1) = P(100) + P(101) P(110) - P(111) = 0.125 + 0.125 + 0.125 - 0.125 = 0.25 \\
Re[\gamma_{11}] = 2(C(0) - C(1)) = 0
$$

Actually I'm not sure if the imaginary part has the exact same post-processing; it's not completely obvious from the paper.

Nevertheless, that's not $\frac{1}{\sqrt{2}}$

In [33]:
gamma([], 0, 0)

tensor(0.+2.22044605e-16j, requires_grad=True)

This should have outputted $\frac{1}{\sqrt{2}}$.